In [4]:
import sys
import os
import time
import json


import numpy as np
import sklearn
import pandas as pd

from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
import matplotlib.pyplot as plt


from sklearn.preprocessing import MinMaxScaler
from sklearn.metrics import mean_absolute_error
from sklearn.preprocessing import KBinsDiscretizer
from sklearn.manifold import TSNE

from sklearn import tree

Matplotlib is building the font cache; this may take a moment.


In [48]:
url = 'https://raw.githubusercontent.com/malogalbo/BasicTitanic/main/data/test.csv'
test_data = pd.read_csv(url, encoding='cp1252', index_col = 0, on_bad_lines = "skip")
url = 'https://raw.githubusercontent.com/malogalbo/BasicTitanic/main/data/train.csv'
train_data = pd.read_csv(url, encoding='cp1252', index_col = 0, on_bad_lines = "skip")

# Loading and Pre-Processing Data

In [10]:
#Main Cell

[[0 3 'Braund, Mr. Owen Harris' ... 7.25 nan 'S']
 [1 1 'Cumings, Mrs. John Bradley (Florence Briggs Thayer)' ... 71.2833
  'C85' 'C']
 [1 3 'Heikkinen, Miss. Laina' ... 7.925 nan 'S']
 ...
 [0 3 'Johnston, Miss. Catherine Helen "Carrie"' ... 23.45 nan 'S']
 [1 1 'Behr, Mr. Karl Howell' ... 30.0 'C148' 'C']
 [0 3 'Dooley, Mr. Patrick' ... 7.75 nan 'Q']]


In [11]:
def train_val_split(x, y, prop_vec, shuffle=True, seed=None):

    assert x.shape[0] == y.shape[0]
    prop_vec = prop_vec / np.sum(prop_vec) # normalize

    n = x.shape[0]
    n_train = int(np.ceil(n * prop_vec[0]))
    n_val = n - n_train

    assert np.amin([n_train, n_val]) >= 1   

    if shuffle:
        rng = np.random.default_rng(seed)
        pi = rng.permutation(n)
    else:
        pi = np.arange(0, n)

    pi_train = pi[0:n_train]
    pi_val = pi[n_train:n]

    train_x = x[pi_train]
    train_y = y[pi_train]

    val_x = x[pi_val]
    val_y = y[pi_val]  
    
    return train_x, train_y, val_x, val_y

One Hot Encode:
"Pclass" 3rd, "Embarked" 

From String to Bool:
"Sex" 5th

Normalize/Standardize
"Fare"
"Age"
"Parch"
"sibsp"


Ignore/Rid 
"Name" 4th
PassengerID 1st
"ticket"


Perhaps?
"Cabin"


y-column:
"Survived" 2nd

In [12]:
# Clean According To Above

# Extract y-column from training

# Then Split

In [49]:
#Drop Name, Cabin, Ticket
names = ['Name', 'Cabin', 'Ticket']
train_data = train_data.drop(columns=names, axis=1)
test_data = test_data.drop(columns=names, axis=1)
print(train_data.info())

<class 'pandas.core.frame.DataFrame'>
Int64Index: 891 entries, 1 to 891
Data columns (total 8 columns):
 #   Column    Non-Null Count  Dtype  
---  ------    --------------  -----  
 0   Survived  891 non-null    int64  
 1   Pclass    891 non-null    int64  
 2   Sex       891 non-null    object 
 3   Age       714 non-null    float64
 4   SibSp     891 non-null    int64  
 5   Parch     891 non-null    int64  
 6   Fare      891 non-null    float64
 7   Embarked  889 non-null    object 
dtypes: float64(2), int64(4), object(2)
memory usage: 62.6+ KB
None


In [50]:
#extract y
train_y = train_data.loc[:, "Survived"]
train_x = train_data.drop(columns=['Survived'], axis=1)

#no test y
test_x = test_data

print(train_y)

PassengerId
1      0
2      1
3      1
4      1
5      0
      ..
887    0
888    1
889    0
890    1
891    0
Name: Survived, Length: 891, dtype: int64


In [51]:
test_x = test_x.to_numpy()
train_x = train_x.to_numpy()
train_y = train_y.to_numpy()



In [52]:
print(train_x[0])
print(test_x[0])

[3 'male' 22.0 1 0 7.25 'S']
[3 'male' 34.5 0 0 7.8292 'Q']


In [53]:
#one hot encode categorical variables
from sklearn.preprocessing import OneHotEncoder
encoder = OneHotEncoder(handle_unknown='ignore')

features_to_standardize = [0, 6] #pclass and embarked

#train_x
tmp = encoder.fit_transform(train_x[:, features_to_standardize]).toarray() #encode data

train_x = np.delete(train_x, features_to_standardize, 1) #delete all old cat columns, 1 for col
train_x = np.concatenate((train_x, tmp), axis = 1) #add encoded variables to train_x

#test_x
tmp = encoder.fit_transform(test_x[:, features_to_standardize]).toarray()

test_x = np.delete(test_x, features_to_standardize, 1) #delete all old cat columns, 1 for col
test_x = np.concatenate((test_x, tmp), axis = 1)

In [54]:
print(train_x[0])
print(test_x[0])

['male' 22.0 1 0 7.25 0.0 0.0 1.0 0.0 0.0 1.0 0.0]
['male' 34.5 0 0 7.8292 0.0 0.0 1.0 0.0 1.0 0.0]


In [55]:
#convert sex from string to bool
for i, sex in enumerate(train_x.T[0]): 
    if (sex == "male"):
        train_x[i, 0] = 1
    else: 
        train_x[i, 0] = 0
for i, sex in enumerate(test_x.T[0]): 
    if (sex == "male"):
        test_x[i, 0] = 1
    else: 
        test_x[i, 0] = 0


In [56]:
print(train_x[0])

[1 22.0 1 0 7.25 0.0 0.0 1.0 0.0 0.0 1.0 0.0]
